In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, SimpleRNN, InputLayer, LSTM, Concatenate
from keras.utils import to_categorical, plot_model
from keras.datasets import mnist
from keras.utils.vis_utils import model_to_dot


# Load dataset from mnist 
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Pre-process data
num_labels = len(np.unique(y_train))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

image_size = x_train.shape[1]
input_size = image_size * image_size


x_train = np.reshape(x_train, [-1, input_size])
x_test = np.reshape(x_test, [-1, input_size])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

input_shape = (image_size, image_size)
batch_size = 128
hidden_units = 512
dropout = 0.45
units = 256

# Define fully connected network/MLP
def mlp_model(input_size=input_size):
  model = Sequential()
  model.add(Dense(hidden_units, input_dim=input_size))
  model.add(Activation('relu'))
  model.add(Dropout(dropout))
  model.add(Dense(hidden_units))
  model.add(Activation('relu'))
  model.add(Dropout(dropout))
  model.add(Dense(hidden_units))
  model.add(Activation('relu'))
  model.add(Dropout(dropout))
  model.add(Dense(hidden_units))
  model.add(Activation('relu'))
  model.add(Dropout(dropout))
  model.add(Dense(num_labels))
  model.add(Activation('softmax'))
  model.summary()
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

plot_model(mlp_model(), to_file='mlp-mnist.png', show_shapes=True)

# Define LSTM network
def lstm_model(units=units, input_shape=input_shape):
  model = Sequential()
  model.add(LSTM(units=units, input_shape=input_shape, return_sequences=True))
  model.add(LSTM(units=units, dropout=dropout, return_sequences=True))
  model.add(LSTM(units=units, dropout=dropout, return_sequences=False))
  model.add(Dense(num_labels))
  model.add(Activation('softmax'))
  model.summary()
  model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

plot_model(lstm_model(), to_file='rnn-mnist.png', show_shapes=True)


# Combine MLP/LSTM models
ensamble_model = []
ensamble_model.append(mlp_model())
ensamble_model.append(lstm_model())


# Train combined MLP/LSTM model
ensamble_models = []
for i in range(len(ensamble_model)):
  if ensamble_model[0]:
    ensamble_model[0].fit(x_train, y_train, epochs=5, batch_size=batch_size)
    ensamble_models.append(ensamble_model[i])
  else:
    ensamble_model[1].fit(x_train, y_train, epochs=1, batch_size=batch_size)
    ensamble_models.append(ensamble_model[i])

# Evaluate Ensamble Model
loss, acc = ensamble_model[(len(ensamble_models) - 2)].evaluate(x_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))


11493376/11490434 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________